In [2]:
postgres_host="localhost"
postgres_port=5432
postgres_user="root"
postgres_password= "root"
postgres_db = 'bank_db'

import psycopg2
from psycopg2.extras import DictCursor


In [3]:
def connect_to_db() -> psycopg2.extensions.connection:
    """
    Connects to the database using the provided credentials.

    Returns:
        psycopg2.extensions.connection: A connection object representing the database connection.
    """
    engine = psycopg2.connect(dbname=postgres_db,
                            user=postgres_user,
                            password=postgres_password,
                            host=postgres_host,
                            port=postgres_port)
    
    return engine


def transform_additional(additional):
    if isinstance(additional, str):
        return (additional,)
    else:
        return additional


In [4]:
def query_for_informations(request_to_do:str=None, additional=None) -> dict:
    """
    
    """
    additional = transform_additional(additional)

    # Initialize query
    if request_to_do == 'get_username_informations':
        query = "SELECT * FROM users WHERE username=%s"

    if request_to_do == 'get_existing_accounts':
        query = 'SELECT * FROM accounts'

    if request_to_do == 'get_existing_budgets':
        query = 'SELECT * FROM budgets'

    if request_to_do == 'get_existing_transactions':
        query = 'SELECT * FROM transactions'



    # Get engine
    engine = connect_to_db()


    # Apply Query
    with engine as conn:
        try:
            with conn.cursor(cursor_factory=DictCursor) as cur:
                try:
                    cur.execute(query, additional)
                    results = cur.fetchall()
                    print(f'results: {results}')
                    if results == None:
                        return {}
                    return results


                except psycopg2.OperationalError as e:
                    print(f"Could not execute the query. Error: {e}")

        except psycopg2.OperationalError as e:
            print(f"Could not connect to the database. Error: {e}")


In [5]:
def query_insert_values(request_to_do:str=None, additional=None) -> None:
    """
    
    """
    additional = transform_additional(additional)

    # initialize query
    if request_to_do == 'create_new_account':
        query = 'INSERT INTO accounts (id, name, type, balance, owner, history) VALUES (%s, %s, %s, %s, %s, %s)'
    if request_to_do == 'delete_account':
        query = 'DELETE FROM accounts WHERE id=%s'

    if request_to_do == 'create_new_budget':
        query = 'INSERT INTO budgets (id, name, month, amount, history) VALUES (%s, %s, %s, %s, %s)'
    if request_to_do == 'delete_budget':
        query = 'DELETE FROM budgets WHERE id=%s'

    if request_to_do == 'create_new_transaction':
        query = 'INSERT INTO transactions (id, date, type, amount, origin_account, destination_account, budget, category, description) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'
    if request_to_do == 'apply_transaction_to_budget':
        query = """
        UPDATE budgets 
        SET amount = amount - %s, history = jsonb_insert(history, '{transaction_ids}', to_jsonb(%s::text), true) 
        WHERE id = %s
        """
    if request_to_do == 'apply_transaction_to_accounts':
        # tuple to list
        additional_list = list(additional)
        # transaction type & remove it from additional
        type = additional[0]
        additional_list.pop(0)

        if type == 'credit':
            # Remove origin account from additional and convert back to tuple
            additional_list.pop(2)
            additional = tuple(additional_list)
            query = """
            UPDATE accounts
            SET balance = balance + %s, history = jsonb_insert(history, '{transaction_ids}', to_jsonb(%s::text), true)
            WHERE id = %s
            """
        if type == 'debit':
            # Remove destination account from additional and convert back to tuple
            additional_list.pop(3)
            additional = tuple(additional_list)
            query = """
            UPDATE accounts
            SET balance = balance - %s, history = jsonb_insert(history, '{transaction_ids}', to_jsonb(%s::text), true)
            WHERE id = %s
            """
        if type == 'transfert':
            # Extract values from additional
            transaction_amount, transaction_id, origin_account, destination_account = additional

            # Query to decrease balance of origin account and add transaction id to history
            query1 = """
            UPDATE accounts
            SET balance = balance - %s, history = jsonb_insert(history, '{transaction_ids}', to_jsonb(%s::text), true)
            WHERE id = %s
            """ % (transaction_amount, transaction_id, origin_account)

            # Query to increase balance of destination account and add transaction id to history
            query2 = """
            UPDATE accounts
            SET balance = balance + %s, history = jsonb_insert(history, '{transaction_ids}', to_jsonb(%s::text), true)
            WHERE id = %s
            """ % (transaction_amount, transaction_id, destination_account)

            # Combine queries
            query = query1 + query2

    # Get engine
    engine = connect_to_db()


    # Apply Query
    with engine as conn:
        try:
            with conn.cursor(cursor_factory=DictCursor) as cur:
                try:
                    cur.execute(query, additional)
                    conn.commit()

                except psycopg2.OperationalError as e:
                    print(f"Could not execute the query. Error: {e}")

        except psycopg2.OperationalError as e:
            print(f"Could not connect to the database. Error: {e}")

In [7]:
request_to_do = 'get_existing_budgets'
additional = None

results = query_for_informations(request_to_do=request_to_do, additional=additional)
default_budget_id = str([budget[0] for budget in results if budget[1] == 'budget_test_transactions'][0])
default_budget_id

results: [['24549c65-f223-4090-bc92-d3b7107d8635', 'default', 'N/A', 0.0, {}, datetime.datetime(2024, 6, 6, 7, 19, 50, 985282), datetime.datetime(2024, 6, 6, 7, 19, 50, 985282)], ['2d74549c-6d9c-4ccb-8a82-c3f8bae3b73a', 'budget_test_transactions', 'June', 10000000.0, {}, datetime.datetime(2024, 6, 6, 15, 35, 50, 390358), datetime.datetime(2024, 6, 6, 15, 35, 50, 390358)]]


'2d74549c-6d9c-4ccb-8a82-c3f8bae3b73a'

In [60]:
existing_budgets = [result[1:3] for result in results]
existing_budgets

[]